In [1]:
import findspark

In [2]:
findspark.init('C:\spark')
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *

In [3]:
spark = SparkSession \
        .builder \
        .master("local")\
        .appName("USVehicleAccidentAnalysis") \
        .getOrCreate()

In [8]:
df_charges = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Charges_use.csv", header=True)
df_damages = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Damages_use.csv", header=True)
df_units = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Units_use.csv", header=True)
df_endorse = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Endorse_use.csv", header=True)
df_restrict = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Restrict_use.csv", header=True)
df_primary_person_use = spark.read.csv("D:/bcg-accident-casestudy/app/Data/Primary_Person_use.csv", header=True)

In [5]:
df_charges.describe()
#df_damages.describe()
#df_endorse.describe()
#df_restrict.describe()

DataFrame[summary: string, CRASH_ID: string, UNIT_NBR: string, PRSN_NBR: string, CHARGE: string, CITATION_NBR: string]

In [12]:
df_charges.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_charges.columns]
   ).show()
df_damages.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_damages.columns]
   ).show()
df_units.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_units.columns]
   ).show()
df_primary_person_use.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_primary_person_use.columns]
   ).show()

+--------+--------+--------+------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE|CITATION_NBR|
+--------+--------+--------+------+------------+
|       0|       0|       0|   104|        7359|
+--------+--------+--------+------+------------+

+--------+----------------+
|CRASH_ID|DAMAGED_PROPERTY|
+--------+----------------+
|       0|               6|
+--------+----------------+

+--------+--------+------------+-------------+----------+----------------+----+------------+------------+-----------+----------+----------------+---------------+--------+-----------------+----------------+------------------+-----------------+--------------+------------------+-----------------+--------------+------------------+---------------+---------------+------------------+------------------+-------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR|UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR

In [36]:
## df_units analysis 
# df_units.filter(col('CRASH_ID')=='15019844').distinct().show()
# df_primary_person_use.filter(col('CRASH_ID')=='15019844').show()
#print(df_units.count())
#print(df_units.distinct().count())
#173499
#168124
## DUPLICATES
#print(df_units.drop('VEH_DMAG_AREA_1_ID','VEH_DMAG_AREA_2_ID').distinct().count())
### UNIQUE UNITS
#print(df_units.select('CRASH_ID','UNIT_NBR','VIN').distinct().count())
#df_units.distinct().groupBy('CRASH_ID','UNIT_NBR','VIN').agg(count('CRASH_ID').alias('count')) \
#    .filter(col('count')>1).show()
#df_units.filter(col('VIN')=='1C6RR7FT7DS652168').distinct().show(truncate=False)
# print(df_units.select('CRASH_ID','UNIT_NBR','VIN').distinct().count())
# print(df_units.select('CRASH_ID','UNIT_NBR').distinct().count())
# print(df_units.select('CRASH_ID','VIN').distinct().count())
#### SAME VEHICLES BUT DIFFERENT MANUFACTURERS ??##########################
df_units.filter(col('VIN')=='1GCEK19V12E132663').show()
df_units.select('CRASH_ID','UNIT_NBR','VIN').filter(col('VIN').isNotNull()) \
        .distinct().groupBy('CRASH_ID','VIN').count().filter(col('count')>1).orderBy(col('count')).show(10000)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+-----------+----------+----------------+---------------+--------+-----------------+--------------------+--------------------+-----------------+--------------+------------------+-----------------+--------------+------------------+---------------+---------------+--------------------+------------------+--------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR| UNIT_DESC_ID|VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|              VIN|VEH_MOD_YEAR|VEH_COLOR_ID|VEH_MAKE_ID|VEH_MOD_ID|VEH_BODY_STYL_ID|EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|    FIN_RESP_TYPE_ID|  VEH_DMAG_AREA_1_ID|VEH_DMAG_SCL_1_ID|FORCE_DIR_1_ID|VEH_DMAG_AREA_2_ID|VEH_DMAG_SCL_2_ID|FORCE_DIR_2_ID|VEH_INVENTORIED_FL|VEH_TRANSP_NAME|VEH_TRANSP_DEST|  CONTRIB_FACTR_1_ID|CONTRIB_FACTR_2_

In [8]:
# df_primary_person_use analysis
print(df_primary_person_use.columns)
print(df_primary_person_use.count())
print(df_primary_person_use.distinct().count())
print(df_primary_person_use.select('CRASH_ID','UNIT_NBR').distinct().count())
print(list(set(df_units.columns) & set(df_primary_person_use.columns)))
# common columns CRASH_ID, UNIT_NBR

['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'PRSN_TYPE_ID', 'PRSN_OCCPNT_POS_ID', 'PRSN_INJRY_SEV_ID', 'PRSN_AGE', 'PRSN_ETHNICITY_ID', 'PRSN_GNDR_ID', 'PRSN_EJCT_ID', 'PRSN_REST_ID', 'PRSN_AIRBAG_ID', 'PRSN_HELMET_ID', 'PRSN_SOL_FL', 'PRSN_ALC_SPEC_TYPE_ID', 'PRSN_ALC_RSLT_ID', 'PRSN_BAC_TEST_RSLT', 'PRSN_DRG_SPEC_TYPE_ID', 'PRSN_DRG_RSLT_ID', 'DRVR_DRG_CAT_1_ID', 'PRSN_DEATH_TIME', 'INCAP_INJRY_CNT', 'NONINCAP_INJRY_CNT', 'POSS_INJRY_CNT', 'NON_INJRY_CNT', 'UNKN_INJRY_CNT', 'TOT_INJRY_CNT', 'DEATH_CNT', 'DRVR_LIC_TYPE_ID', 'DRVR_LIC_STATE_ID', 'DRVR_LIC_CLS_ID', 'DRVR_ZIP']
156954
156954
156954
['UNIT_NBR', 'INCAP_INJRY_CNT', 'NONINCAP_INJRY_CNT', 'NON_INJRY_CNT', 'POSS_INJRY_CNT', 'DEATH_CNT', 'UNKN_INJRY_CNT', 'TOT_INJRY_CNT', 'CRASH_ID']


In [9]:
# df_charges analysis
print(df_charges.columns)
print(df_charges.count())
print(df_charges.distinct().count())
print(df_charges.select('CRASH_ID','UNIT_NBR').distinct().count())
print(df_charges.select('CRASH_ID','UNIT_NBR', 'PRSN_NBR').distinct().count())
print(df_charges.select('CRASH_ID','UNIT_NBR', 'PRSN_NBR', 'CHARGE').distinct().count())
df_charges.show(truncate=False)

['CRASH_ID', 'UNIT_NBR', 'PRSN_NBR', 'CHARGE', 'CITATION_NBR']
116110
115909
93299
93415
115842
+--------+--------+--------+----------------------------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                  |CITATION_NBR|
+--------+--------+--------+----------------------------------------+------------+
|14768622|1       |1       |DRIVING WHILE INTOXICATED               |NULL        |
|14838637|1       |1       |DWI                                     |1600000015  |
|14838641|1       |1       |RAN RED LIGHT SOLID (TC 544.007)        |L20440      |
|14838641|2       |1       |NO DRIVER'S LICENSE (TC521.025)         |L23141      |
|14838668|1       |1       |DRIVING WHILE INTOXICATED               |TX4IC50SRJD3|
|14838669|2       |1       |DWI W/BAC >.015                         |2015-000006 |
|14838670|1       |1       |DRIVING WHILE INTOXICATED               |2016-000003 |
|14838685|1       |1       |FAILED TO DRIVE SINGLE LANE, NOL, NO INS|13843

In [10]:
# df_damages analysis
print(df_damages.count())
print(df_damages.select('CRASH_ID').distinct().count())
print(df_damages.distinct().count())
df_damages.groupBy('CRASH_ID','DAMAGED_PROPERTY')\
        .agg(count('CRASH_ID').alias('ID_COUNTS'))\
        .filter(col('ID_COUNTS')>1)\
        .join(df_damages,['CRASH_ID','DAMAGED_PROPERTY'],'inner')\
        .orderBy(col('CRASH_ID')).show(10,False)
# Duplicates exists
df_damages.select('DAMAGED_PROPERTY').filter(upper(col('DAMAGED_PROPERTY')).like('N%')).distinct().show(20000,False)
# no damage- NONE, NONE1

24950
20887
24606
+--------+------------------------+---------+
|CRASH_ID|DAMAGED_PROPERTY        |ID_COUNTS|
+--------+------------------------+---------+
|14846500|METAL FENCE & MAILBOX   |2        |
|14846500|METAL FENCE & MAILBOX   |2        |
|14851892|FENCE                   |2        |
|14851892|FENCE                   |2        |
|14852708|SPORTSMAN TRAVEL TRAILER|2        |
|14852708|SPORTSMAN TRAVEL TRAILER|2        |
|14852740|ROADWAY SIGN            |2        |
|14852740|ROADWAY SIGN            |2        |
|14855146|UTILITY POLE            |2        |
|14855146|UTILITY POLE            |2        |
+--------+------------------------+---------+
only showing top 10 rows

+----------------------------------------+
|DAMAGED_PROPERTY                        |
+----------------------------------------+
|NO STANDING STOPPING PARKING SIGN       |
|NO U TURN SIGN                          |
|NEWSPAPER BOX                           |
|NO HEAVY TRUCK SIGN                     |
|NETWIRE FE

In [11]:
# 1. Analytics 1: Find the number of crashes (accidents) in which number of males killed are greater than 2
# Dataframes: df_primary_person_use

In [18]:
total_count = df_primary_person_use.select('CRASH_ID','PRSN_TYPE_ID','PRSN_GNDR_ID').count()
total_distinct_count = df_primary_person_use.select('CRASH_ID','PRSN_GNDR_ID').distinct().count()
print(total_count,total_distinct_count)
df_primary_person_use.select('PRSN_GNDR_ID').distinct().show()
print(df_primary_person_use.count())
print(df_primary_person_use.distinct().count())
print(df_primary_person_use.select('CRASH_ID','UNIT_NBR').distinct().count())

156954 114949
+------------+
|PRSN_GNDR_ID|
+------------+
|          NA|
|     UNKNOWN|
|        MALE|
|      FEMALE|
+------------+

156954
156954
156954


In [30]:
# df_primary_person unique along units involved(CRASH_ID,UNIT_NBR)
df_primary_person_male = df_primary_person_use.filter(col('PRSN_GNDR_ID')=='MALE')
df_primary_person_male_count = df_primary_person_male.groupBy('CRASH_ID') \
    .agg(count('CRASH_ID').alias('MALE_COUNT')) \
    .filter(col('MALE_COUNT')>2)
df_primary_person_male_count.show()
print('df_primary_person_male_count',df_primary_person_male_count.count())

Py4JJavaError: An error occurred while calling o693.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 300.0 failed 1 times, most recent failure: Lost task 0.0 in stage 300.0 (TID 174) (DESKTOP-9G6ITOO executor driver): org.apache.spark.SparkFileNotFoundException: File file:/D:/bcg-accident-casestudy/app/Data/Primary_Person_use.csv does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:780)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:220)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:279)
	at org.ap

In [14]:
# Analysis 2: How many two wheelers are booked for crashes?

In [15]:
df_units.select("VEH_BODY_STYL_ID").distinct().show(truncate=False)

+---------------------------------+
|VEH_BODY_STYL_ID                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+



In [16]:
df_units_cleaned = df_units.select('CRASH_ID','UNIT_NBR','VIN','VEH_BODY_STYL_ID').distinct()
df_two_wheelers = df_units_cleaned.filter(col("VEH_BODY_STYL_ID").contains("MOTORCYCLE"))
two_Wheelers_count = df_two_wheelers.count()
df_two_wheelers.show(5)
print(two_Wheelers_count)

+--------+--------+-----------------+----------------+
|CRASH_ID|UNIT_NBR|              VIN|VEH_BODY_STYL_ID|
+--------+--------+-----------------+----------------+
|14982941|       1|1HD1BJY145Y017506|      MOTORCYCLE|
|15051763|       2|JKAEXEE16GDA21497|      MOTORCYCLE|
|15738225|       2|1HD4CAM10JY121226|      MOTORCYCLE|
|14882525|       1|        9D38345H6|      MOTORCYCLE|
|15443852|       2|1HD1KBM1XCB666237|      MOTORCYCLE|
+--------+--------+-----------------+----------------+
only showing top 5 rows

773


In [24]:
# Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.
# df_primary_person_use-> PRSN_DEATH_TIME,PRSN_AIRBAG_ID,CRASH_ID
# df_units-> VEH_MAKE_ID,CRASH_ID
#df_units.filter(col('VEH_MAKE_ID')=='NA').distinct().show(1000,False)

df_primary_person_use.select('PRSN_TYPE_ID').distinct().show(1000,False)
df_primary_person_use.select('PRSN_AIRBAG_ID').distinct().show()

df_primary_person_use.filter(col('CRASH_ID')=='15146770').select('CRASH_ID','UNIT_NBR').show()
df_units.filter(col('CRASH_ID')=='15146770').select('CRASH_ID','UNIT_NBR').show()
df_units.select('VEH_MAKE_ID').distinct().show(1000)

df_pp_airbag_failure_and_death = df_primary_person_use.filter(col('PRSN_DEATH_TIME').isNotNull() \
                                                    & (col('PRSN_AIRBAG_ID') == 'NOT DEPLOYED') \
                                                    & (col('PRSN_TYPE_ID').contains('DRIVER')))
df_units_cleaned = df_units.select('CRASH_ID','UNIT_NBR','VIN','VEH_MAKE_ID').distinct()
df_top_make = df_units_cleaned.filter(col('VEH_MAKE_ID')!='NA').join(df_pp_airbag_failure_and_death, ['CRASH_ID','UNIT_NBR'], 'inner') \
    .groupby("VEH_MAKE_ID").agg(count('VEH_MAKE_ID').alias('COUNT_OF_DRIVER_DEATH_AND_AB_FAILURE')) \
    .orderBy(col("COUNT_OF_DRIVER_DEATH_AND_AB_FAILURE").desc()).limit(5)
df_top_make.show()
print({row['VEH_MAKE_ID']:row['COUNT_OF_DEATH_AND_ABFAILURE'] for row in df_top_make.collect()})

+---------------------------------+
|PRSN_TYPE_ID                     |
+---------------------------------+
|PEDESTRIAN                       |
|DRIVER OF MOTORCYCLE TYPE VEHICLE|
|PASSENGER/OCCUPANT               |
|DRIVER                           |
|UNKNOWN                          |
|OTHER (EXPLAIN IN NARRATIVE)     |
|PEDALCYCLIST                     |
+---------------------------------+

+-----------------+
|   PRSN_AIRBAG_ID|
+-----------------+
|               NA|
|   DEPLOYED, REAR|
|     NOT DEPLOYED|
|   DEPLOYED, SIDE|
|          UNKNOWN|
|   NOT APPLICABLE|
|DEPLOYED MULTIPLE|
|  DEPLOYED, FRONT|
+-----------------+

+--------+--------+
|CRASH_ID|UNIT_NBR|
+--------+--------+
|15146770|       1|
|15146770|       2|
|15146770|       3|
|15146770|       4|
|15146770|       5|
|15146770|       6|
|15146770|       7|
|15146770|       8|
|15146770|       9|
|15146770|      10|
|15146770|      11|
|15146770|      12|
|15146770|      13|
|15146770|      14|
+--------+--------+

+

PySparkValueError: COUNT_OF_DEATH_AND_ABFAILURE

In [25]:
# Analysis 4: Determine number of Vehicles with driver having valid licences involved in hit and run?
# conditions: valid licence, hit and run
# valid license: 
# hit and run: df_units-> VEH_HNR_FL
# df_primary_person_use.select('DRVR_LIC_TYPE_ID', 'DRVR_LIC_CLS_ID').distinct().orderBy(col('DRVR_LIC_TYPE_ID')).show(10000)
# df_primary_person_use.select('DRVR_LIC_TYPE_ID').distinct().show(truncate=False)
# df_units.select('VEH_HNR_FL').distinct().show()
df_hit_n_run = df_units.filter(col('VEH_HNR_FL')=='Y') \
                .select('CRASH_ID','UNIT_NBR','VIN','VEH_HNR_FL').distinct()

df_valid_license = df_primary_person_use.filter(~col('DRVR_LIC_CLS_ID').isin(['UNKNOWN','NA','UNLICENSED']) \
                            & (col('DRVR_LIC_TYPE_ID').isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."])))

df_result = df_valid_license.join(df_hit_n_run, ['CRASH_ID','UNIT_NBR'],'inner')
# df_result.show()
print(df_result.count())

2536


In [38]:
# Analysis 5: Which state has highest number of accidents in which females are not involved?
df_primary_person_use.select('PRSN_GNDR_ID').distinct().show()
df_primary_person_use.select('DRVR_LIC_STATE_ID').distinct().show(1000)
df_crash_with_females = df_primary_person_use.filter(col('PRSN_GNDR_ID') == "FEMALE") \
            .select('CRASH_ID').distinct()
df_no_female_crash = df_primary_person_use \
            .filter(~lower(col('DRVR_LIC_STATE_ID')).isin(['na','unknown', 'other'])) \
            .join(df_crash_with_females, ['CRASH_ID'], 'leftanti') \
            .select('CRASH_ID','DRVR_LIC_STATE_ID').distinct()
df_result = df_no_female_crash.groupby("DRVR_LIC_STATE_ID") \
            .agg(count('DRVR_LIC_STATE_ID').alias('NO_FEMALE_ACCIDENT_COUNT')) \
            .orderBy(col("NO_FEMALE_ACCIDENT_COUNT").desc())
print(df_result.first().DRVR_LIC_STATE_ID)

+------------+
|PRSN_GNDR_ID|
+------------+
|          NA|
|     UNKNOWN|
|        MALE|
|      FEMALE|
+------------+

+--------------------+
|   DRVR_LIC_STATE_ID|
+--------------------+
|                Utah|
|              Hawaii|
| U.S. Virgin Islands|
|           Minnesota|
|                Ohio|
|Northern Mariana ...|
|            Arkansas|
|              Oregon|
|                  NA|
|               Texas|
|        North Dakota|
|        Pennsylvania|
|         Connecticut|
|            Nebraska|
|             Vermont|
|              Nevada|
|         Puerto Rico|
|          Washington|
|            Illinois|
|            Oklahoma|
|             Unknown|
|District of Columbia|
|               Other|
|            Delaware|
|American Samoa (I...|
|              Alaska|
|          New Mexico|
|       West Virginia|
|            Missouri|
|        Rhode Island|
|              Mexico|
|             Georgia|
|             Montana|
|            Michigan|
|            Virginia|
|    

In [39]:
# Analysis 6: Which are the Top 3rd to 5th VEH_MAKE_IDs that contribute to a largest number of injuries including death

df_units_cleaned = df_units.filter(col('VEH_MAKE_ID') != "NA") \
        .select('CRASH_ID','UNIT_NBR','VIN','VEH_MAKE_ID','TOT_INJRY_CNT','DEATH_CNT').distinct()
df = df_units_cleaned.groupby('VEH_MAKE_ID').agg(sum('TOT_INJRY_CNT').alias('AGG_INJURIES'), sum('DEATH_CNT').alias('AGG_DEATHS')) \
                    .withColumn('AGG_COUNT', col('AGG_INJURIES')+col('AGG_DEATHS')) \
                    .orderBy(col("AGG_COUNT").desc()) \
                    .limit(5)
df2 = df.withColumn("id", monotonically_increasing_id()).filter(col("id") >= 2).drop("id")
df.show()
print([company_row['VEH_MAKE_ID'] for company_row in df2.collect()])


+-----------+------------+----------+---------+
|VEH_MAKE_ID|AGG_INJURIES|AGG_DEATHS|AGG_COUNT|
+-----------+------------+----------+---------+
|  CHEVROLET|      6963.0|      37.0|   7000.0|
|       FORD|      6901.0|      42.0|   6943.0|
|     TOYOTA|      4207.0|      20.0|   4227.0|
|      DODGE|      3124.0|      12.0|   3136.0|
|     NISSAN|      3092.0|      21.0|   3113.0|
+-----------+------------+----------+---------+

['TOYOTA', 'DODGE', 'NISSAN']


In [42]:
# Analysis 7: For all the body styles involved in crashes, mention the top ethnic user group of each unique body style
df_units.select('VEH_BODY_STYL_ID').distinct().show(truncate=False)
df_primary_person_use.select('PRSN_ETHNICITY_ID').distinct().show(truncate=False)
undefined_body_values = ["NOT REPORTED","OTHER  (EXPLAIN IN NARRATIVE)", "UNKNOWN","NA"]

df_units_cleaned = df_units.filter(~col('VEH_BODY_STYL_ID').isin(undefined_body_values)) \
        .select('CRASH_ID','UNIT_NBR','VIN','VEH_HNR_FL','VEH_BODY_STYL_ID') \
        .distinct()
df_primary_person_cleaned = df_primary_person_use.filter(~col('PRSN_ETHNICITY_ID').isin(["NA", "UNKNOWN", "OTHER"]))

window = Window.partitionBy("VEH_BODY_STYL_ID")
df_result = df_units_cleaned.join(df_primary_person_cleaned, on=['CRASH_ID','UNIT_NBR'], how='inner') \
    .groupby("VEH_BODY_STYL_ID", "PRSN_ETHNICITY_ID").count() \
    .withColumn("rn", row_number().over(window.orderBy(col("count").desc()))) \
    .filter(col("rn") == 1) \
    .drop("rn", "count")
df_result.show()

+---------------------------------+
|VEH_BODY_STYL_ID                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+

+---------------------------+
|PRSN_ETHNICITY_ID          |
+---------------------------+
|WHITE                      |
|BLACK         

In [22]:
# Analysis 8: Among the crashed cars, what are the Top 5 Zip Codes with highest number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)
# CONTRIB_FACTR_1_ID, CONTRIB_FACTR_P1_ID defined to be same but have different values


df_units_cleaned =df_units.filter(lower(col('CONTRIB_FACTR_1_ID')).contains('alcohol') \
                | lower(col('CONTRIB_FACTR_2_ID')).contains('alcohol') \
                | lower(col('CONTRIB_FACTR_P1_ID')).contains('alcohol')) \
        .select('CRASH_ID','UNIT_NBR','VIN','VEH_HNR_FL','VEH_BODY_STYL_ID') \
        .distinct()
df_primary_person_cleaned = df_primary_person_use.na.drop(subset = ['DRVR_ZIP'])
df_alc_crash = df_units_cleaned.join(df_primary_person_cleaned, on=['CRASH_ID','UNIT_NBR'], how='inner') \
        .groupBy('DRVR_ZIP').agg(count('DRVR_ZIP').alias('ALCOHOL_CRASH_COUNT')) \
        .orderBy(col("ALCOHOL_CRASH_COUNT").desc()) \
        .limit(5)
df_alc_crash.show()
print([zip_row['DRVR_ZIP'] for zip_row in df_alc_crash.collect()])

+--------+-------------------+
|DRVR_ZIP|ALCOHOL_CRASH_COUNT|
+--------+-------------------+
|   76010|                 52|
|   78521|                 47|
|   75067|                 41|
|   78130|                 41|
|   78542|                 38|
+--------+-------------------+

['76010', '78521', '75067', '78130', '78542']


In [23]:
# Analysis 9: Count of Distinct Crash IDs where No Damaged Property was observed and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance
# Remove integers and leave NA as it might stand for North america
df_damages_unique = df_damages.filter(upper(col('DAMAGED_PROPERTY')).isin(['NONE', 'NONE1'])) \
        .select('CRASH_ID') \
        .distinct()
insurance_claims = ['INSURANCE BINDER', 'LIABILITY INSURANCE POLICY', 'PROOF OF LIABILITY INSURANCE']
df_units_cleaned = df_units.filter(
            (
                ((~col('VEH_DMAG_SCL_1_ID').isin(["NA", "NO DAMAGE", "INVALID VALUE"]))\
                    & (col('VEH_DMAG_SCL_1_ID') >= 'DAMAGED 5')) | \
                ((~col('VEH_DMAG_SCL_2_ID').isin(["NA", "NO DAMAGE", "INVALID VALUE"]))\
                    & (col('VEH_DMAG_SCL_2_ID') >= 'DAMAGED 5'))) \
            & col('FIN_RESP_TYPE_ID').isin(insurance_claims)
        ) \
        .select('CRASH_ID','UNIT_NBR','VIN','VEH_DMAG_SCL_1_ID', \
                        'VEH_DMAG_SCL_2_ID','FIN_RESP_TYPE_ID') \
        .distinct()

df_crash_id_result = df_units_cleaned.join(df_damages_unique, ['CRASH_ID'], 'inner')
print([crash_id_row['CRASH_ID'] for crash_id_row in df_crash_id_result.collect()])



['15232090', '14885395', '14870169', '15255328', '15249931', '14894076', '14996273', '15307513', '15232090', '14885395', '14870169']


In [24]:
df_units.select('VEH_DMAG_SCL_1_ID').distinct().show(1000)
df_units.select('VEH_DMAG_SCL_2_ID').distinct().show(1000)
df_units.select('FIN_RESP_TYPE_ID').distinct().show(1000,False)

+-----------------+
|VEH_DMAG_SCL_1_ID|
+-----------------+
|        DAMAGED 4|
|    INVALID VALUE|
|               NA|
|        DAMAGED 5|
|DAMAGED 1 MINIMUM|
|        DAMAGED 3|
|        NO DAMAGE|
|DAMAGED 7 HIGHEST|
|        DAMAGED 2|
|        DAMAGED 6|
+-----------------+

+-----------------+
|VEH_DMAG_SCL_2_ID|
+-----------------+
|        DAMAGED 4|
|    INVALID VALUE|
|               NA|
|        DAMAGED 5|
|DAMAGED 1 MINIMUM|
|        DAMAGED 3|
|        NO DAMAGE|
|DAMAGED 7 HIGHEST|
|        DAMAGED 2|
|        DAMAGED 6|
+-----------------+

+----------------------------------------+
|FIN_RESP_TYPE_ID                        |
+----------------------------------------+
|INSURANCE BINDER                        |
|LIABILITY INSURANCE POLICY              |
|NA                                      |
|CERTIFICATE OF SELF-INSURANCE           |
|CERTIFICATE OF DEPOSIT WITH COUNTY JUDGE|
|CERTIFICATE OF DEPOSIT WITH COMPTROLLER |
|SURETY BOND                             |
|PROOF O

In [25]:
# 0. Analysis 10: Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers,
# used top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)
df_units_cleaned = df_units.select('CRASH_ID','UNIT_NBR','VIN','VEH_LIC_STATE_ID','VEH_COLOR_ID','VEH_MAKE_ID').distinct()
df_top_25_states = df_units_cleaned.filter(col("VEH_LIC_STATE_ID").cast("float").isNull()) \
        .groupBy('VEH_LIC_STATE_ID').count() \
        .orderBy(col('count').desc()).limit(25)
df_top_25_states.show()

df_top_10_colors = df_units_cleaned.filter((col('VEH_COLOR_ID') != "NA") \
                                           & (col("VEH_COLOR_ID").cast("float").isNull())) \
            .groupby("VEH_COLOR_ID").count() \
            .orderBy(col("count").desc()).limit(10)
df_top_10_colors.show()

df_pp_licenced = df_primary_person_use.filter(col('DRVR_LIC_TYPE_ID').isin(["DRIVER LICENSE", "COMMERCIAL DRIVER LIC."]))
df_speed_charges = df_charges.filter(col('CHARGE').contains('SPEED') | col('CHARGE').contains('FAST'))\
            .select('CRASH_ID', 'UNIT_NBR').distinct()

df_result = df_units_cleaned.join(df_speed_charges, ['CRASH_ID', 'UNIT_NBR'], 'inner') \
                    .join(df_pp_licenced, ['CRASH_ID', 'UNIT_NBR'], 'inner') \
                    .join(df_top_10_colors, ['VEH_COLOR_ID'], 'inner') \
                    .join(df_top_25_states, ['VEH_LIC_STATE_ID'], 'inner') \
                    .groupby("VEH_MAKE_ID").count() \
                    .orderBy(col("count").desc()).limit(5)
df_result.show()
print([result_row['VEH_MAKE_ID'] for result_row in df_result.collect()])

+----------------+------+
|VEH_LIC_STATE_ID| count|
+----------------+------+
|              TX|153408|
|              NA|  6031|
|              UN|  1400|
|              OK|   761|
|              LA|   718|
|              NM|   705|
|              MX|   409|
|              CA|   391|
|              FL|   371|
|              AR|   318|
|              IN|   306|
|              IL|   255|
|              TN|   233|
|              AZ|   215|
|              MS|   204|
|              CO|   168|
|              GA|   160|
|              KS|   152|
|              MO|   142|
|              NC|   134|
+----------------+------+
only showing top 20 rows

+------------+-----+
|VEH_COLOR_ID|count|
+------------+-----+
|         WHI|35745|
|         BLK|27494|
|         SIL|20714|
|         GRY|18049|
|         BLU|14889|
|         RED|13535|
|         GRN| 6612|
|         MAR| 5892|
|         TAN| 4796|
|         GLD| 4028|
+------------+-----+

+-----------+-----+
|VEH_MAKE_ID|count|
+-----------+--

In [26]:
print(list(set(df_charges.columns) & set(df_units.columns)))
df_units.select("VEH_LIC_STATE_ID").distinct().show(2000,False)
df_units.filter(col("VEH_LIC_STATE_ID").cast("int").isNull()).select("VEH_LIC_STATE_ID").distinct().show()
print(df_units.filter(col("VEH_LIC_STATE_ID").cast("float").isNull()).count())
df_units.select("VEH_COLOR_ID").distinct().show(2000,False)

['UNIT_NBR', 'CRASH_ID']
+----------------+
|VEH_LIC_STATE_ID|
+----------------+
|AZ              |
|SC              |
|LA              |
|MN              |
|NJ              |
|MX              |
|DC              |
|OR              |
|NA              |
|98              |
|VA              |
|RI              |
|KY              |
|WY              |
|NH              |
|MI              |
|NV              |
|WI              |
|ID              |
|CA              |
|CT              |
|NE              |
|MT              |
|NC              |
|VT              |
|MD              |
|DE              |
|MO              |
|IL              |
|ME              |
|WA              |
|ND              |
|US              |
|MS              |
|AL              |
|IN              |
|OH              |
|TN              |
|NM              |
|IA              |
|PA              |
|SD              |
|NY              |
|UN              |
|TX              |
|WV              |
|DS              |
|GA              |
|MA   

In [27]:
df_primary_person_use.select('DRVR_LIC_TYPE_ID') .distinct().show()
df_charges.select('CHARGE').filter(col('CHARGE').contains('RUN')).distinct().show(1000,False)

+--------------------+
|    DRVR_LIC_TYPE_ID|
+--------------------+
|                  NA|
|COMMERCIAL DRIVER...|
|             ID CARD|
|             UNKNOWN|
|        OCCUPATIONAL|
|          UNLICENSED|
|               OTHER|
|      DRIVER LICENSE|
+--------------------+

+-------------------------------------------------------+
|CHARGE                                                 |
+-------------------------------------------------------+
|UNSAFE SPEED, RUN RED LIGHT, FTLI, FAIL REPORT ACCIDENT|
|HIT AND RUN $200                                       |
|ACCIDENT INVOLVING DAMAGE TO VEHICLE (HIT AND RUN)     |
|HIT AND RUN OVER $200                                  |
|NO DL, FMFR, RUN RED LIGHT                             |
|RUN RED LIGHT                                          |
|TC 544.007 RUN RED LIGHT                               |
|HIT AND RUN MORE THAN $200                             |
|FAIL TO YIELD ROW - LEFT TRUN                          |
|RUNNING A STOP SIGN       

In [28]:
# print(df_units.select('CRASH_ID','UNIT_NBR','VIN','VEH_LIC_STATE_ID','VEH_COLOR_ID','VEH_MAKE_ID').distinct().count())
# print(df_units.distinct().count())
# print(df_units.count())